## Azure ML - Sample Batch Prediction Pipeline
This notebook demonstrates creation and execution of an Azure ML pipeline designed to load data from a remote source, to make predictions against that data using a previously registered ML model, and finally save that data to an external sink where it can be consumed by other LOB apps. 

### Import Required Packages

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData, PipelineEndpoint
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig

### Connect to Azure ML Workspace, Provision Compute Resources, and get References to Datastores
Connect to workspace using config associated config file. Get a reference to you pre-existing AML compute cluster or provision a new cluster to facilitate processing. Finally, get references to your default blob datastore.

In [ ]:
#Connect to AML Workspace
ws = Workspace.from_config()

#Select AML Compute Cluster
cpu_cluster_name = 'cpucluster'

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           min_nodes=0,
                                                           max_nodes=1)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)
    
#Get default datastore
default_ds = ws.get_default_datastore()

 ### Create Run Configuration
The `RunConfiguration` defines the environment used across all python steps. You can optionally add additional conda or pip packages to be added to your environment. [More details here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py).

In [ ]:
run_config = RunConfiguration()
run_config.docker.use_docker = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['requests', 'pandas'])
run_config.environment.python.conda_dependencies.add_pip_package('snowflake-connector-python[pandas]')
run_config.environment.python.conda_dependencies.add_pip_package('scikit-learn==0.24')

### Define Output Datasets
Below we define the configuration for datasets that will be passed between steps in our pipeline. Note, in all cases we specify the datastore that should hold the datasets and whether they should be registered following step completion or not. This can optionally be disabled by removing the register_on_complete() call.

In [ ]:
inferencing_dataset = OutputFileDatasetConfig(name='inferencing_dataset', destination=(default_ds, 'inferencing_dataset/{run-id}')).read_delimited_files().register_on_complete(name='inferencing_data')
scored_dataset = OutputFileDatasetConfig(name='scored_dataset', destination=(default_ds, 'scored_dataset/{run-id}')).read_delimited_files().register_on_complete(name='scored_data')

### Define Pipeline Parameters
PipelineParameter objects serve as variable inputs to an Azure ML pipeline and can be specified at runtime. Below we specify a pipeline parameter object `model_name` which will be used to reference the locally trained model that was uploaded and registered within the Azure ML workspace. Multiple pipeline parameters can be created and used. Included here are multiple sample pipeline parameters (`get_data_param_*`) to highlight how parameters can be passed into and consumed by various pipeline steps.  

In [ ]:
model_name = PipelineParameter(name='model_name', default_value='iris_classification')
get_data_param_1 = PipelineParameter(name='get_data_param_1', default_value='value_1')
get_data_param_2 = PipelineParameter(name='get_data_param_2', default_value='value_2')
get_data_param_3 = PipelineParameter(name='get_data_param_3', default_value='value_3')

### Define Pipeline Steps
The pipeline below consists of steps to gather and register data from a remote source, a scoring step where the registered model is used to make predictions on loaded, and a data publish step where scored data can be exported to a remote data source. All of the PythonScriptSteps have a corresponding *.py file which is referenced in the step arguments. Also, any PipelineParameters defined above can be passed to and consumed within these steps.

In [ ]:
get_inferencing_data_step = PythonScriptStep(
    name='Get Inferencing Data',
    script_name='get_inferencing_data.py',
    arguments=[
        '--get_data_param_1', get_data_param_1,
        '--get_data_param_2', get_data_param_2,
        '--get_data_param_3', get_data_param_3,
        '--inferencing_dataset', inferencing_dataset
    ],
    outputs=[inferencing_dataset],
    compute_target=cpu_cluster,
    source_directory='.',
    allow_reuse=False,
    runconfig=run_config
)

score_inferencing_data_step = PythonScriptStep(
    name='Score Inferencing Data',
    script_name='score_inferencing_data.py',
    arguments=[
        '--model_name', model_name,
        '--scored_dataset', scored_dataset
    ],
    inputs=[inferencing_dataset.as_input(name='inferencing_data')],
    outputs=[scored_dataset],
    compute_target=cpu_cluster,
    source_directory='.',
    allow_reuse=False,
    runconfig=run_config
)

publish_scored_data_step = PythonScriptStep(
    name='Publish Scored Data',
    script_name='publish_scored_data.py',
    inputs=[scored_dataset.as_input(name='scored_data')],
    compute_target=cpu_cluster,
    source_directory='.',
    allow_reuse=False,
    runconfig=run_config
)

### Create Pipeline
Create an Azure ML Pipeline by specifying the steps to be executed. Note: based on the dataset dependencies between steps, exection occurs logically such that no step will execute unless all of the necessary input datasets have been generated.

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[get_inferencing_data_step, score_inferencing_data_step, publish_scored_data_step])

### Create Experiment and Run Pipeline
Define a new experiment (logical container for pipeline runs) and execute the pipeline. You can modify the values of pipeline parameters here when submitting a new run.

In [ ]:
experiment = Experiment(ws, 'iris_batch_predictions')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)

### Publish Pipeline
Create a published version of your pipeline that can be triggered via a REST API call.

In [ ]:
published_pipeline = pipeline.publish(name = 'Iris Batch Prediction Pipeline',
                                     description = 'Pipeline that generates batch predictions using a locally trained model.',
                                     continue_on_step_failure = False)